# Sistema di Raccomandazione basato su Similarità Semantica

Questo notebook mostra un esempio pratico di recommendation system basato su embedding semantici.
L'obiettivo è suggerire articoli alternativi o simili a partire dalla descrizione di un articolo dato.

---

## Alternative tradizionali per la ricerca di articoli simili

Prima dell’avvento degli embedding semantici, i sistemi di raccomandazione testuale spesso usavano metodi come:

- **Keyword Matching**  
  Ricerca basata sulla presenza di parole chiave identiche tra query e documenti.  
  Svantaggi:  
  - Sensibile a sinonimi e variazioni linguistiche  
  - Non cattura il significato reale del testo  
  - Risultati poco rilevanti se la formulazione cambia anche leggermente

- **TF-IDF (Term Frequency - Inverse Document Frequency)**  
  Pesa le parole in base alla loro frequenza nel documento e nella collezione, migliorando il keyword matching.  
  Svantaggi:  
  - Ancora basato su parole esatte, poco efficace con sinonimi o espressioni diverse  
  - Non tiene conto del contesto o della semantica del testo  
  - Funziona male con testi brevi o poco strutturati

---

### Vantaggi dell’approccio basato su embedding semantici

- **Comprende il significato e il contesto** del testo, non solo la presenza di parole  
- Gestisce sinonimi e variazioni linguistiche automaticamente  
- Funziona bene anche con testi brevi o incompleti  
- Permette di calcolare una **similarità continua** e più fine tra articoli, non solo match esatti  
- Si presta facilmente a integrazioni con modelli di AI e sistemi scalabili

---

In sintesi, gli embedding semantici permettono un salto di qualità nel matching testuale, offrendo risultati più pertinenti e naturali rispetto ai metodi tradizionali basati su keyword o TF-IDF.


In [ ]:
import warnings

import faiss
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [ ]:
df = pd.DataFrame(
    [
        {
            "codice": "A1001",
            "descrizione": "Valvola in acciaio inox da 1 pollice",
            "categoria": "valvole",
        },
        {
            "codice": "A1002",
            "descrizione": "Valvola in ottone da 1 pollice",
            "categoria": "valvole",
        },
        {
            "codice": "A1003",
            "descrizione": "Rubinetto a sfera per impianti termici",
            "categoria": "rubinetti",
        },
        {
            "codice": "A1004",
            "descrizione": "Valvola a farfalla con leva manuale",
            "categoria": "valvole",
        },
        {
            "codice": "A1005",
            "descrizione": "Raccordo in PVC per tubi da 50mm",
            "categoria": "raccordi",
        },
        {
            "codice": "A1006",
            "descrizione": "Valvola regolatrice in acciaio per alta pressione",
            "categoria": "valvole",
        },
        {
            "codice": "A1007",
            "descrizione": "Rubinetto con attacco rapido",
            "categoria": "rubinetti",
        },
        {
            "codice": "A1008",
            "descrizione": "Tubo in rame flessibile da 3 metri",
            "categoria": "tubi",
        },
        {
            "codice": "A1009",
            "descrizione": "Raccordo in ottone per tubo flessibile",
            "categoria": "raccordi",
        },
        {
            "codice": "A1010",
            "descrizione": "Valvola di ritegno per impianto idraulico",
            "categoria": "valvole",
        },
    ]
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["descrizione"])

query_descrizione = df.loc[df["codice"] == "A1001", "descrizione"].values[0]
query_tfidf = vectorizer.transform([query_descrizione])

In [ ]:
similarity_tfidf = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
top_tfidf_idx = similarity_tfidf.argsort()[::-1][1:4]

df[["codice", "descrizione", "categoria"]].loc[top_tfidf_idx.tolist()]

## Cosa sono gli embedding?

Un embedding è una rappresentazione numerica (vettore) di un oggetto, in questo caso la **descrizione di un articolo**.

Gli embedding trasformano testo in punti in uno spazio multidimensionale. Articoli con descrizioni simili vengono mappati su **vettori vicini tra loro**.

Nel nostro esempio, ogni descrizione diventa un vettore di centinaia di dimensioni (es. 1536 se si usa `text-embedding-ada-002`).

Per visualizzarli useremo una **riduzione dimensionale** per proiettare i punti in 2D.

In [ ]:
index = faiss.read_index("database.index")
n = index.ntotal
embeddings = np.vstack([index.reconstruct(i) for i in range(n)])
df["embedding"] = list(embeddings)
embeddings_matrix = np.vstack(df["embedding"].values)

In [ ]:
df["embedding"].iloc[0]

In [ ]:
len(df["embedding"].iloc[0])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
embedding_2d = pca.fit_transform(embeddings_matrix)

plt.figure(figsize=(10, 6))
for i, row in df.iterrows():
    x, y = embedding_2d[i]
    plt.scatter(x, y, marker="o", color="blue")
    plt.text(x + 0.01, y + 0.01, row["codice"], fontsize=9)

plt.title("Proiezione 2D degli articoli nello spazio degli embedding")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

## Similarità tra vettori

Nel nostro modello, articoli simili hanno **vettori di embedding vicini** nello spazio.

Per misurare la similarità usiamo la **cosine similarity** (o, nel caso di FAISS L2, la distanza euclidea):
- Similarità alta = angolo piccolo → articoli semanticamente affini
- Similarità bassa = angolo grande → articoli diversi

Nell'immagine sopra, gli articoli vicini tra loro sono considerati alternative plausibili.

## Cosine Similarity vs L2 Similarity (distanza euclidea)

Quando confrontiamo vettori (come gli embedding), possiamo misurare quanto sono "simili" usando metriche diverse:

- **Cosine Similarity**: misura l'angolo tra due vettori. Indica quanto sono orientati nella stessa direzione, indipendentemente dalla loro lunghezza.
  - Valori tra -1 e 1.
  - 1 = vettori perfettamente allineati (massima similarità).
  - 0 = vettori ortogonali (nessuna similarità).
  - -1 = vettori opposti.

- **L2 Similarity** (o distanza euclidea): misura la distanza "lineare" tra due punti nello spazio.
  - Più la distanza è piccola, più i vettori sono vicini.
  - La distanza è sempre ≥ 0.

In generale:

- La cosine similarity si concentra sull'**orientamento** del vettore.
- La L2 similarity si concentra sulla **distanza spaziale**.

---

Nei motori di ricerca vettoriali come FAISS, a seconda del tipo di indice, si usa una o l'altra: 
- Indici basati su distanza L2 cercano i vettori più **vicini** spazialmente.
- Indici basati su cosine similarity cercano i vettori con angoli più **piccoli**.

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def l2_distance(a, b):
    return np.linalg.norm(a - b)


# Vettori di esempio (2D per visualizzazione)
v1 = np.array([1, 0])
v2 = np.array([0.5, 0.5])
v3 = np.array([-1, 0])

print("Cosine similarity v1-v2:", cosine_similarity(v1, v2))
print("L2 distance v1-v2:", l2_distance(v1, v2))
print("Cosine similarity v1-v3:", cosine_similarity(v1, v3))
print("L2 distance v1-v3:", l2_distance(v1, v3))

# Visualizzazione
origin = np.array([0, 0])

plt.figure(figsize=(6, 6))
plt.quiver(*origin, *v1, color="r", scale=3, label="v1")
plt.quiver(*origin, *v2, color="g", scale=3, label="v2")
plt.quiver(*origin, *v3, color="b", scale=3, label="v3")
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.grid(True)
plt.legend()
plt.title("Visualizzazione vettori e loro angoli")
plt.show()

In [ ]:
def raccomanda_simili(codice_articolo: str, top_n: int = 5):
    articolo = df[df["codice"] == codice_articolo]
    if articolo.empty:
        raise ValueError("Codice articolo non trovato")

    query_embedding = np.array(articolo["embedding"].iloc[0]).reshape(1, -1)
    distanze, indici = index.search(query_embedding, top_n + 1)

    risultati = df.iloc[indici[0]]
    risultati["similarità"] = np.exp(-distanze[0])
    risultati = risultati[risultati["codice"] != codice_articolo]
    return risultati[["codice", "descrizione", "categoria", "similarità"]].sort_values(
        by="similarità", ascending=False
    )

In [ ]:
raccomanda_simili("A1001", top_n=3)

In [ ]:
def mostra_simili_2d(codice_articolo: str, top_n: int = 5):
    articolo_idx = df.index[df["codice"] == codice_articolo].item()
    query_emb = embeddings_matrix[articolo_idx].reshape(1, -1)
    _, indici = index.search(query_emb, top_n + 1)

    plt.figure(figsize=(10, 6))
    for i in range(len(df)):
        x, y = embedding_2d[i]
        if i == articolo_idx:
            plt.scatter(
                x, y, marker="x", color="red", s=100, label="Articolo richiesto"
            )
        elif i in indici and i != articolo_idx:
            plt.scatter(
                x,
                y,
                marker="o",
                color="green",
                s=80,
                label="Simile"
                if "Simile" not in plt.gca().get_legend_handles_labels()[1]
                else "",
            )
        else:
            plt.scatter(x, y, marker="o", color="lightgray")

        plt.text(x + 0.01, y + 0.01, df.iloc[i]["codice"], fontsize=8)

    plt.title(f"Articolo '{codice_articolo}' e simili nello spazio 2D")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
mostra_simili_2d("A1001", top_n=3)

# Conclusioni

Abbiamo costruito un sistema di recommendation che:

1. Usa **embedding semantici** di Azure OpenAI per rappresentare articoli come vettori numerici.
2. Indica articoli simili cercandoli in uno spazio vettoriale con **FAISS**.
3. Mostra un'alternativa moderna a TF-IDF e keyword matching, capace di cogliere la **semantica**.

---